## Install package

In [ ]:
pip install 'aif360[all]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.1/214.1 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 46.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 KB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━

## Import packages

In [ ]:
import numpy as np
import pandas as pd
import copy

from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing.lfr import LFR

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

## Define *compute_metric()* function

In [ ]:
# Metrics function
from collections import OrderedDict
from aif360.metrics import ClassificationMetric

def compute_metrics(dataset_true, dataset_pred, 
                    unprivileged_groups, privileged_groups,
                    disp = True):
    """ Compute the key metrics """
    classified_metric_pred = ClassificationMetric(dataset_true,
                                                 dataset_pred, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    metrics = OrderedDict()
    metrics["Balanced accuracy"] = 0.5*(classified_metric_pred.true_positive_rate()+
                                             classified_metric_pred.true_negative_rate())
    metrics["Statistical parity difference"] = classified_metric_pred.statistical_parity_difference()
    metrics["Disparate impact"] = classified_metric_pred.disparate_impact()
    metrics["Average odds difference"] = classified_metric_pred.average_odds_difference()
    metrics["Equal opportunity difference"] = classified_metric_pred.equal_opportunity_difference()
    metrics["Theil index"] = classified_metric_pred.theil_index()
    
    if disp:
        for k in metrics:
            print("%s = %.4f" % (k, metrics[k]))
    
    return metrics

## Get dataset, encode string variables, and split into training and testing sets

In [ ]:
# Get the dataset and split into train and test
df_unprocessed = pd.read_csv("https://raw.githubusercontent.com/oliver-miller/ECSE557/master/A3/heart_assignment3.csv")

df = df_unprocessed.copy()

# Define the headers of the data to facilitate code at the end of the cell (when splitting into X and y data)
headers = df.columns.to_list()

# Encode string parameters as integers
# Will facilitate the data analysis and binary classifier
# Male = 1, female = 0 (for privileged and unprivileged groups)
df['Sex'] = df['Sex'].replace("M", 1)
df['Sex'] = df['Sex'].replace("F", 0)
df['ExerciseAngina'] = df['ExerciseAngina'].replace("N", 0)
df['ExerciseAngina'] = df['ExerciseAngina'].replace("Y", 1)
df['ChestPainType'] = df['ChestPainType'].replace("ASY", 0)
df['ChestPainType'] = df['ChestPainType'].replace("NAP", 1)
df['ChestPainType'] = df['ChestPainType'].replace("TA", 2)
df['ChestPainType'] = df['ChestPainType'].replace("ATA", 3)
df['RestingECG'] = df['RestingECG'].replace("Normal", 0)
df['RestingECG'] = df['RestingECG'].replace("LVH", 1)
df['RestingECG'] = df['RestingECG'].replace("ST", 2)
df['ST_Slope'] = df['ST_Slope'].replace("Flat", 0)
df['ST_Slope'] = df['ST_Slope'].replace("Down", 1)
df['ST_Slope'] = df['ST_Slope'].replace("Up", 2)
# White = 1, everyone else = 0 (for privileged and unprivileged groups)
df['Race'] = df['Race'].replace("White", 1)
df['Race'] = df['Race'].replace("Black", 0)
df['Race'] = df['Race'].replace("Asian", 0)
df['Race'] = df['Race'].replace("Hispanic ", 0)
df['Race'] = df['Race'].replace("Other", 0)

X = df.drop(columns=['HeartDisease'], inplace=False)
y = df['HeartDisease']

# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True)

## Fit and predict for standard LogisticRegression classifier

In [ ]:
# Fit and predict with logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Calculate and print accuracy
accuracy = np.sum(y_test == y_pred) / len(y_pred)
print("ACCURACY: " + str(accuracy))

ACCURACY: 0.8745874587458746


# 4. [PA] For the classifier that you created in last assignment (the non-private classifier), use AIF 360 to set-up and calculate three different fairness metrics. Calculate these metrics for different groups based on the groups you identified as privileged and unprivileged in step 2.

Define privileged and unprivileged groups

In [ ]:
privileged_groups_sex = [{'Sex': 1}]
unprivileged_groups_sex = [{'Sex': 0}]

privileged_groups_race = [{'Race': 1}]
unprivileged_groups_race = [{'Race': 0}]

Fairess metrics on training data

In [ ]:
# Format training dataset to use package functions
X_train_fair = X_train.copy(deep=True)
y_train_fair = y_train.copy(deep=True)
df_train_fair = pd.concat([X_train_fair, y_train_fair], axis=1)

# Create binary label datasets for sex and race groups
df_train_fair_BLD_sex = BinaryLabelDataset(
    df=df_train_fair, 
    label_names=["HeartDisease"], 
    protected_attribute_names=["Sex"],
    favorable_label=1.0,
    unfavorable_label=0.0
    )
df_train_fair_BLD_race = BinaryLabelDataset(
    df=df_train_fair, 
    label_names=["HeartDisease"], 
    protected_attribute_names=["Race"],
    favorable_label=1.0,
    unfavorable_label=0.0
    )

# Create binary label dataset metrics for sex and race groups
metric_train_sex = BinaryLabelDatasetMetric(df_train_fair_BLD_sex, 
                                                 unprivileged_groups=unprivileged_groups_sex,
                                                 privileged_groups=privileged_groups_sex)
metric_train_race = BinaryLabelDatasetMetric(df_train_fair_BLD_race, 
                                                  unprivileged_groups=unprivileged_groups_race,
                                                  privileged_groups=privileged_groups_race)

# Print results
print("SEX")
print("Privileged group: Male, unprivileged group: female")
print("Statistical Parity Difference: {}".format(metric_train_sex.statistical_parity_difference()))
print("Disparate Impact: {}".format(metric_train_sex.disparate_impact()))

print("\n")

print("RACE")
print("Privileged group: White, unprivileged group: black, asian, hispanic, other")
print("Statistical Parity Difference: {}".format(metric_train_race.statistical_parity_difference()))
print("Disparate Impact: {}".format(metric_train_race.disparate_impact()))

SEX
Privileged group: Male, unprivileged group: female
Statistical Parity Difference: -0.3594907407407408
Disparate Impact: 0.4190048634493079


RACE
Privileged group: White, unprivileged group: black, asian, hispanic, other
Statistical Parity Difference: -0.05821676078028748
Disparate Impact: 0.900643394934976


Fairness metrics on testing data

In [ ]:
# Format testing dataset to use package functions
y_pred_Series = pd.Series(y_pred, name="HeartDisease", index=y_test.index)
df_test_fair = pd.concat([X_test, y_test], axis=1)
df_pred_fair = pd.concat([X_test, y_pred_Series], axis=1)

# Create binary label datasets for sex and race groups
# Test data
df_test_fair_BLD_sex = BinaryLabelDataset(
    df=df_test_fair, 
    label_names=["HeartDisease"], 
    protected_attribute_names=["Sex"],
    favorable_label=1.0,
    unfavorable_label=0.0
    )
df_test_fair_BLD_race = BinaryLabelDataset(
    df=df_test_fair, 
    label_names=["HeartDisease"], 
    protected_attribute_names=["Race"],
    favorable_label=1.0,
    unfavorable_label=0.0
    )
# Predicted data
df_pred_fair_BLD_sex = BinaryLabelDataset(
    df=df_pred_fair, 
    label_names=["HeartDisease"], 
    protected_attribute_names=["Sex"],
    favorable_label=1.0,
    unfavorable_label=0.0
    )
df_pred_fair_BLD_race = BinaryLabelDataset(
    df=df_pred_fair, 
    label_names=["HeartDisease"], 
    protected_attribute_names=["Race"],
    favorable_label=1.0,
    unfavorable_label=0.0
    )

# Create binary label dataset metrics for sex and race groups
# Sex
print("SEX")
sex_fairness_metrics = compute_metrics(df_test_fair_BLD_sex, 
                                       df_pred_fair_BLD_sex, 
                                       unprivileged_groups_sex, 
                                       privileged_groups_sex,
                                       disp=False)
print("\n")
# Race
print("RACE")
race_fairness_metrics = compute_metrics(df_test_fair_BLD_race, 
                                        df_pred_fair_BLD_race, 
                                        unprivileged_groups_race, 
                                        privileged_groups_race,
                                        disp=False)

SEX
OrderedDict([('Balanced accuracy', 0.8734117752326414),
             ('Statistical parity difference', -0.5101337086558762),
             ('Disparate impact', 0.23323460968902054),
             ('Average odds difference', -0.3111825316577591),
             ('Equal opportunity difference', -0.5254658385093167),
             ('Theil index', 0.09210864395493067)])


RACE
OrderedDict([('Balanced accuracy', 0.8734117752326414),
             ('Statistical parity difference', -0.125),
             ('Disparate impact', 0.8125),
             ('Average odds difference', -0.047495277983082826),
             ('Equal opportunity difference', -0.09196025293586263),
             ('Theil index', 0.09210864395493067)])


# 7. [PA][DA] Prioritize has now asked you to do some pre-processing to minimize unfairness for TriageAssist. Using AIF 360, implement one pre-processing mitigation technique covered in class.

In [ ]:
# Set seed for consistency between runs (facilitates debugging)
seed = 0

# Learning fair representations dataset for sex and race groups   
TR_sex = LFR(unprivileged_groups=unprivileged_groups_sex,
             privileged_groups=privileged_groups_sex,
             k=10, Ax=0.1, Ay=1.0, Az=2.0,
             verbose=0,
             seed=seed
        )

 
TR_race = LFR(unprivileged_groups=unprivileged_groups_race,
              privileged_groups=privileged_groups_race,
              k=10, Ax=0.1, Ay=1.0, Az=2.0,
              verbose=0,
              seed=seed
        )

# Fit and transform sex and race groups
TR_sex = TR_sex.fit(df_train_fair_BLD_sex, maxiter=5000, maxfun=5000)
df_train_fair_BLD_sex_TR = TR_sex.transform(df_train_fair_BLD_sex)

TR_race = TR_race.fit(df_train_fair_BLD_race, maxiter=5000, maxfun=5000)
df_train_fair_BLD_race_TR = TR_race.transform(df_train_fair_BLD_race)

# Create binary label dataset metrics for sex and race groups (tranformed)
metric_train_sex_TR = BinaryLabelDatasetMetric(df_train_fair_BLD_sex_TR, 
                                                 unprivileged_groups=unprivileged_groups_sex,
                                                 privileged_groups=privileged_groups_sex)
metric_train_race_TR = BinaryLabelDatasetMetric(df_train_fair_BLD_race_TR, 
                                                  unprivileged_groups=unprivileged_groups_race,
                                                  privileged_groups=privileged_groups_race)

# Print results
print("SEX")
print("Privileged group: Male, unprivileged group: female")
print("Statistical Parity Difference: {}".format(metric_train_sex_TR.statistical_parity_difference()))
print("Disparate Impact: {}".format(metric_train_sex_TR.disparate_impact()))

print("\n")

print("RACE")
print("Privileged group: White, unprivileged group: black, asian, hispanic, other")
print("Statistical Parity Difference: {}".format(metric_train_race_TR.statistical_parity_difference()))
print("Disparate Impact: {}".format(metric_train_race_TR.disparate_impact()))

SEX
Privileged group: Male, unprivileged group: female
Statistical Parity Difference: 0.0
Disparate Impact: 1.0


RACE
Privileged group: White, unprivileged group: black, asian, hispanic, other
Statistical Parity Difference: -0.0771143480492813
Disparate Impact: 0.700889801505818
